# Contour Tests

In [1]:
# Type imports
from typing import Any, Dict, List, Tuple

# Standard Libraries
from enum import Enum, auto
from pathlib import Path
from collections import Counter, defaultdict
from dataclasses import dataclass, field, asdict
from math import sqrt, pi, sin, cos, tan, radians
from math import isnan
from statistics import mean
from itertools import zip_longest

# Shared Packages
import pandas as pd
import numpy as np
import xlwings as xw

import pygraphviz as pgv
import networkx as nx
import matplotlib.pyplot as plt
import shapely

In [2]:
from utilities import *
from types_and_classes import *

In [3]:
%matplotlib inline

In [ ]:
def make_contour_slices(roi_num: ROI_Num, slices: List[SliceIndex],
                        contours: List[Contour]):
    data_list = []
    for slice_idx in slices:
        data_item = {
            'ROI Num': roi_num,
            'Slice Index': SliceIndex(slice_idx),
            'Structure Slice': StructureSlice(contours)
            }
        data_list.append(data_item)
    slice_data = pd.DataFrame(data_list)
    slice_data.set_index(['ROI Num', 'Slice Index'], inplace=True)
    return slice_data

### Make test structures

In [7]:
# 6 cm x 6 cm box
box6 = shapely.Polygon(box_points(6))
slices = make_slice_list(5)
slices_1 = make_contour_slices(1, slices, [box6])

# 4 cm x 4 cm box
box4 = shapely.Polygon(box_points(4))
slices = make_slice_list(5)
#slices = make_slice_list(5, start=0.2, spacing=0.2)
slices_2 = make_contour_slices(2, slices, [box4])

# 6 cm x 6 cm box offset right by 3 cm
offset_box6 = shapely.Polygon(box_points(6, offset_x=3))
slices = make_slice_list(5)
#slices = make_slice_list(5, start=0.2, spacing=0.2)
slices_3 = make_contour_slices(3, slices, [offset_box6])

# 6 cm x 6 cm box with 4cm x 4 cm hole
box6 = shapely.Polygon(box_points(6))
box4 = shapely.Polygon(box_points(4))
slices = make_slice_list(5, start=0.2)
slices_4 = make_contour_slices(4, slices, [box6, box4])

# 2 2x2 boxes
box2a = shapely.Polygon(box_points(2, offset_x=-3))
box2b = shapely.Polygon(box_points(2, offset_x=3))
slices = make_slice_list(5)
#slices = make_slice_list(5, start=0.2, spacing=0.2)
slices_5 = make_contour_slices(5, slices, [box2a, box2b])

# 2 cm x 2 cm box offset inf by 0.3 cm
slices = make_slice_list(5, start=-0.3)
slices_6 = make_contour_slices(6, slices, [offset_box6])


In [8]:
# combine the slice data
slice_data = pd.concat([slices_1, slices_2, slices_3, slices_4, slices_5,
                        slices_6])
# convert slice data into a table of slices and structures
slice_table = make_slice_table(slice_data)

In [9]:
relationship_1_2 = Relationship((1, 2), slice_table)
print(relationship_1_2.relationship_type)

RelationshipType.CONTAINS


In [10]:
relationship_1_2 = Relationship((1, 6), slice_table)
print(relationship_1_2.relationship_type)

RelationshipType.OVERLAPS


In [11]:
relationship_1_2 = Relationship((4, 2), slice_table)
print(relationship_1_2.relationship_type)

RelationshipType.BORDERS
